In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.utils import shuffle

In [2]:
%matplotlib inline
sns.set_style('whitegrid')

In [3]:
df = pd.read_csv('ml-20m/ratings.csv')

In [4]:
df

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944


In [5]:
df.userId = pd.Categorical(df.userId)
df['new_user_id'] = df.userId.cat.codes

In [6]:
df.movieId= pd.Categorical(df.movieId)
df['new_movie_id'] = df.movieId.cat.codes

In [7]:
user_ids = df['new_user_id'].values
movie_ids = df['new_movie_id'].values
ratings = df['rating'].values - 2.5

In [8]:
N = len(set(user_ids))
M = len(set(movie_ids))
D = 10

In [9]:
# MAKE A NEURAL NETWORK
class Model(nn.Module):
    def __init__(self, n_users, n_items, embed_dim, n_hidden=1024):
        super(Model, self).__init__()
        self.N = n_users
        self.M = n_items
        self.D = embed_dim
        
        self.u_emb = nn.Embedding(self.N, self.D)
        self.m_emb = nn.Embedding(self.M, self.D)
        self.fc1 = nn.Linear(2*self.D, n_hidden)
        self.fc2 = nn.Linear(n_hidden, 1)
        
        self.u_emb.weight.data = nn.Parameter(torch.Tensor(np.random.randn(self.N, self.D) * 0.01))
        self.m_emb.weight.data = nn.Parameter(torch.Tensor(np.random.randn(self.M, self.D) * 0.01))
        
    def forward(self, u, m):
        u = self.u_emb(u)
        m = self.m_emb(m)
        
        # merge
        out = torch.cat((u,m), 1)
        
        # ANN
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [10]:
device = torch.device('cuda:0')

In [11]:
model = Model(N, M, D)
model.to(device)

Model(
  (u_emb): Embedding(138493, 10)
  (m_emb): Embedding(26744, 10)
  (fc1): Linear(in_features=20, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1, bias=True)
)

In [14]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=0.08, momentum=0.9)

In [13]:
def batch_gd2(model, criterion, optimizer, train_data, test_data, epochs, bs=512):
    train_users, train_movies, train_ratings = train_data
    test_users, test_movies, test_ratings = test_data
    
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    
    Ntrain = len(train_users)
    batches_per_epoch = int(np.ceil(Ntrain / bs))
    
    for i in range(epochs):
        t0 = datetime.now()
        train_loss = []
        
        train_users, train_movies, train_ratings = shuffle(train_users, train_movies, train_ratings)
        
        for j in range(batches_per_epoch):
            users = train_users[j*bs:(j+1)*bs]
            movies = train_movies[j*bs:(j+1)*bs]
            targets = train_ratings[j*bs:(j+1)*bs]
            
            users = torch.from_numpy(users).long()
            movies = torch.from_numpy(movies).long()
            targets = torch.from_numpy(targets)
            
            targets = targets.view(-1, 1).float()
            
            users, movies, targets = users.to(device), movies.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(users, movies)
            loss = criterion(outputs, targets)
        
            loss.backward()
            optimizer.step()
        
            train_loss.append(loss.item())
            
        train_loss = np.mean(loss)
        
        test_loss = []
        for j in range(int(np.ceil(len(test_users) / bs))):
            users = test_users[j*bs:(j+1)*bs]
            movies = test_movies[j*bs:(j+1)*bs]
            targets = test_ratings[j*bs:(j+1)*bs]
            
            users = torch.from_numpy(users).long()
            movies = torch.from_numpy(movies).long()
            targets = torch.from_numpy(targets)
            
            targets = targets.view(-1, 1).float()
            
            users, movies, targets = users.to(device), movies.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(users, movies)
            loss = criterion(outputs, targets).item()
        
            test_loss.append(loss.item())
            
        test_loss = np.mean(loss)
        
        train_losses[i] = train_loss
        test_losses[i] = test_loss
        
        dt = datetime.now() - t0
        print(f'Epoch {i+1}/{epochs}, Train Loss: {train_loss:.4f},\t Test Loss: {test_loss:.4f}, Duration: {dt}')
        
    return train_losses, test_losses

In [14]:
Ntrain = int(0.8 * len(ratings))
train_users = user_ids[:Ntrain]
train_movies = movie_ids[:Ntrain]
train_ratings = ratings[:Ntrain]

test_users = user_ids[Ntrain:]
test_movies = movie_ids[Ntrain:]
test_ratings = ratings[Ntrain:]

In [16]:
train_losses, test_losses = batch_gd2(model, criterion, optimizer, (train_users, train_movies, train_ratings), (test_users, test_movies, test_ratings), 10)

TypeError: mean() received an invalid combination of arguments - got (axis=NoneType, dtype=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)


In [15]:
# shuffle the data in corresponding order
user_ids, movie_ids, ratings = shuffle(user_ids, movie_ids, ratings)

In [16]:
# convert to tensors
user_ids_t = torch.from_numpy(user_ids).long()
movie_ids_t = torch.from_numpy(movie_ids).long()
ratings_t = torch.from_numpy(ratings)

In [17]:
# Make datasets
Ntrain = int(0.8 * len(ratings))
train_dataset = torch.utils.data.TensorDataset(user_ids_t[:Ntrain], movie_ids_t[:Ntrain], ratings_t[:Ntrain])
test_dataset = torch.utils.data.TensorDataset(user_ids_t[Ntrain:], movie_ids_t[Ntrain:], ratings_t[Ntrain:])

In [18]:
# Data loaders
batch_size = 512
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [19]:
def batch_grad(model, criterion, optimizer, train_iter, test_iter, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    for i in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for users, movies, targets in train_loader:
            targets = targets.view(-1, 1).float()
            users, movies, targets = users.to(device), movies.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(users, movies)
            loss = criterion(outputs, targets)
        
            loss.backward()
            optimizer.step()
        
            train_loss.append(loss.item())
            
        train_loss = np.mean(loss)
        
        test_loss = []
        for users, movies, targets in test_loader:
            targets = targets.view(-1, 1).float()
            optimizer.zero_grad()
            outputs = model(users, movies)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(loss)
        
        train_losses[i] = train_loss
        test_losses[i] = test_loss
        
        dt = datetime.now() - t0
        print(f'Epoch {i+1}/{epochs}, Train Loss: {train_loss:.4f},\t Test Loss: {test_loss:.4f}, Duration: {dt}')
        
    return train_losses, test_losses

In [20]:
# profile this using
%prun train_losses, test_losses = batch_grad(model, criterion, optimizer, train_loader, test_loader, 25)

TypeError: mean() received an invalid combination of arguments - got (axis=NoneType, dtype=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)


In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()

In [ ]:
watched_movie_ids = df[df.new_user_id == 1].new_movie_id.values

In [ ]:
potential_movie_ids = df[~df.new_movie_id.isin(watched_movie_ids)].new_movie_id.unique()

In [ ]:
potential_movie_ids.shape

In [ ]:
len(set(potential_movie_ids))

In [ ]:
user_id_to_recommend = np.ones_like(potential_movie_ids)

In [ ]:
t_user_ids = torch.from_numpy(user_id_to_recommend).long().to(device)
t_movie_ids = torch.from_numpy(potential_movie_ids).long().to(device)

with torch.no_grad():
    predictions = model(t_user_ids, t_movie_ids)

In [ ]:
predictions_np = predictions.cpu().numpy().flatten()

In [ ]:
sort_idx = np.argsort(-predictions_np)

In [ ]:
predictions_np

In [ ]:
sort_idx

In [ ]:
# Top 10 predictions
top_10_movie_ids = potential_movie_ids[sort_idx[:10]]
top_10_scores = predictions_np[sort_idx[:10]]

for movie, score in zip(top_10_movie_ids, top_10_scores):
    print('Movie:', movie, 'Score:', score)